<h1>Запросы SQL из базы данных SQLite (локальный сервер(файл) SQLite )<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подключение-к-базе-данных-с-помощью-Python" data-toc-modified-id="Подключение-к-базе-данных-с-помощью-Python-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подключение к базе данных с помощью Python</a></span></li></ul></div>

## Подключение к базе данных с помощью Python

Для подключения к базе данных SQLite часто используют библиотеку sqlite3 — она одна из самых распространённых. Для подключения к базе данных нам не понадобятся данные для подключения(авторизация)

In [1]:
import sqlite3
import pandas as pd

# Пример подключения к базе данных test_sqlite.db
# Здесь создается подключение к базе данных SQLite с именем файла 'db'. 
# Если файл 'db' не существует, то он будет создан. 
# Если файл существует, то будет установлено соединение с ним.Важно, чтобы файл базы данных, находился в том же катологе 
# исполняемого файла на Python
conn = sqlite3.connect('test_sqlite.db') 

# Создается объект курсора (cursor), который используется для выполнения SQL-запросов в базе данных. 
# Курсор предоставляет методы для выполнения SQL-запросов и получения результатов.Также используется для записи и удаления таблиц 
# в базе данных
cursor = conn.cursor()




Напишем функцию для вывода SQL запроса, где параметром будет являться SQL запрос, а функция вернёт результат заапроса

In [2]:
def select(sql):
  return pd.read_sql(sql,con = conn)

Ранее в базе данных `test_sqlite.db` уже была создана таблица test_1, которую мы создали с помощью IDE DB Browser. Проверим резльтат нашего тестового запроса. 

In [3]:
sql = '''select * from test_1 t '''

In [4]:
select(sql)

,number
0,1
1,2
2,3
3,4
4,5


Результат выполнения успешный

Создадим тестовый датафрейм и зальём его в нашу базу данных **test_sqlite.db**

In [5]:
new_df = pd.DataFrame({'number': [1,2,3,4,5]})
new_df

,number
0,1
1,2
2,3
3,4
4,5


In [6]:
# Параметр if_exists = 'replace' - нужен для того, чтобы перезаписывать таблицу test_1 в базу данных, в случае, если мы перегружаем тетрадку 
# Jupyter, если не поставим, будет ошибка. 
# Параметр index=False, нужен, чтобы в базу данных записалась таблица без индексов

new_df.to_sql('test_1',conn,if_exists = 'replace',index=False)

Выполняем запрос по вновь добавленной в базу данных таблице test_1

In [7]:
sql = '''select number*3 from test_1  '''

In [8]:
select(sql)

,number*3
0,3
1,6
2,9
3,12
4,15


Альтернативный способ вывода sql запроса

In [9]:
pd.read_sql(sql,conn)

,number*3
0,3
1,6
2,9
3,12
4,15


Создадим SQL запрос, где результатом будет новый датафрейм `report_1` и зальём его в нашу базу данных **test_sqlite.db**. DROP TABLE нужно для того, чтобы мы смогли удалить и потом перезаписать нашу созданную ранее таблицу `report_1` в базу данных, в случае перезапуска тетрадки Jupyter. 

In [10]:
# SQL-запрос для удаления таблицы, если она существует
drop_table_query = '''
DROP TABLE IF EXISTS report_1;'''

# SQL-запрос для создания новой таблицы
sql = '''
CREATE TABLE report_1 AS
SELECT number*10 
FROM test_1;
'''

Функция cursor с методом execute используется для записи и удаления таблиц в базе данных

In [11]:
# Выполнение SQL-запроса на удаление созданной ранее таблицы
cursor.execute(drop_table_query)

In [12]:
# Выполнение SQL-запроса на создание новой таблицы из запроса
cursor.execute(sql)

Выполняем запрос по вновь добавленной в базу данных таблице report_1

In [13]:
sql = '''
select *
from report_1
'''

In [14]:
select(sql)

,number*10
0,10
1,20
2,30
3,40
4,50


При завершении работы с базой данных её нужно обязательно сохранить и закрыть

In [15]:
# Сохранение изменений и закрытие соединения
conn.commit()
conn.close()